### WIP

In [2]:
import os
import numpy as np
import pandas as pd 
import pickle
import pymysql
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

### Conectamos con SQL

In [3]:
# Cargar variables de entorno
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("HOST", "localhost")
database = "meneame"

# Crear conexión con SQL
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

### Cargamos df de Clustering para añadir info de Cluster

In [4]:
directorio_base = os.path.abspath(os.path.join(os.getcwd(), "../../.."))  # Subir dos directorios
directorio_pkl = os.path.join(directorio_base, "src", "00.data", "clustering")

# Obtener todos los archivos .pkl en el directorio de preprocesado
archivos_pkl = [f for f in os.listdir(directorio_pkl) if f.startswith("df_clustering_") and f.endswith(".pkl")]

# Si no se encuentran archivos .pkl, mostramos un mensaje
if not archivos_pkl:
    print("❌ No se encontraron archivos .pkl en el directorio.")
else:
    df_lista = []

    # Cargar todos los archivos .pkl
    for archivo in archivos_pkl:
        ruta_archivo = os.path.join(directorio_pkl, archivo)
        try:
            with open(ruta_archivo, "rb") as f:
                df_lista.append(pickle.load(f))
                print(f"✅ Archivo cargado correctamente: {archivo}")
        except FileNotFoundError:
            print(f"❌ No se encontró el archivo: {archivo}")
        except Exception as e:
            print(f"❌ Error al cargar {archivo}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_lista:
        df_clustering = pd.concat(df_lista, ignore_index=True)
        print(f"✅ DataFrame final con {df_clustering.shape[0]} filas y {df_clustering.shape[1]} columnas")
    else:
        print("❌ No se cargaron DataFrames.")

✅ Archivo cargado correctamente: df_clustering_1.pkl
✅ Archivo cargado correctamente: df_clustering_2.pkl
✅ Archivo cargado correctamente: df_clustering_3.pkl
✅ DataFrame final con 287563 filas y 10 columnas


In [5]:
df_clustering.head(3)

,news_id,meneos,clicks,karma,positive_votes,negative_votes,anonymous_votes,comments,category,cluster
0,4032594,115,651,410,57,1,58,30,Política y Sociedad,1
1,4032568,77,982,362,50,2,27,24,Entretenimiento y Cultura,1
2,4032569,97,761,369,50,0,47,70,Política y Sociedad,1


In [6]:
df_clustering['cluster'].value_counts()

cluster
1    234278
0     44330
2      8955
Name: count, dtype: int64

In [7]:
# Crear la tabla de clusters si no existe
create_table_sql = """
CREATE TABLE IF NOT EXISTS cluster_table (
    cluster_id INT PRIMARY KEY,
    cluster_name VARCHAR(50)
);
"""
with engine.connect() as connection:
    connection.execute(text(create_table_sql))

In [8]:
# Insertar nombres de clusters en la tabla
cluster_names = {
    0: "Noticias polémicas",
    1: "Noticias estándar",
    2: "Noticias bien recibidas"
}

with engine.connect() as connection:
    for cluster_id, cluster_name in cluster_names.items():
        insert_sql = text("""
        INSERT INTO cluster_table (cluster_id, cluster_name)
        VALUES (:cluster_id, :cluster_name)
        ON DUPLICATE KEY UPDATE cluster_name = :cluster_name
        """)
        connection.execute(insert_sql, {"cluster_id": cluster_id, "cluster_name": cluster_name})

In [9]:
# Verificar que 'news_id' y 'cluster' están en df_clustering
if not {"news_id", "cluster"}.issubset(df_clustering.columns):
    raise KeyError("Las columnas 'news_id' o 'cluster' no están en df_clustering.")

# Crear diccionario {news_id: cluster_id}
news_cluster_mapping = df_clustering.set_index("news_id")["cluster"].to_dict()

# Asegurar que la columna cluster_id existe en news_info_table
alter_table_sql = """
ALTER TABLE news_info_table 
ADD COLUMN cluster_id INT DEFAULT NULL;
"""
with engine.connect() as connection:
    connection.execute(text(alter_table_sql))

# Actualizar los valores de cluster en la base de datos
update_sql = text("""
    UPDATE news_info_table
    SET cluster_id = :cluster_id
    WHERE news_id = :news_id
""")

with engine.connect() as connection:
    with connection.begin():  # Manejo seguro de la transacción
        for news_id, cluster_id in news_cluster_mapping.items():
            connection.execute(update_sql, {
                "news_id": int(news_id),
                "cluster_id": int(cluster_id)
            })

print("Clusters actualizados en la base de datos.")

Clusters actualizados en la base de datos.


In [10]:
query = text("""
    SELECT cluster_id, COUNT(*) as count 
    FROM news_info_table 
    GROUP BY cluster_id
""")

with engine.connect() as connection:
    result = connection.execute(query)
    cluster_counts_sql = dict(result.fetchall())

print(cluster_counts_sql)

{1: 234278, 2: 8955, 0: 44330}
